# Estimating Velocity from Position

In [1]:
# install dependencies
# !pip install numpy matplotlib

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def kalman_filter(system_initial_values, estimate_and_covar_initial_values, measurements):
    # arrays for tracking
    x_estimates = []
    K_values = []
    P_values = []

    # initialize system variables
    A, H, Q, R = system_initial_values

    # check if shapes of matrices are correct
    assert A.shape[0] == A.shape[1], "A must be square"
    assert A.shape == Q.shape, "A and Q must be same shape"
    assert H.shape[1] == A.shape[1], "H must match state dimensions"


    # Step 0: Set initial values
    x, P = estimate_and_covar_initial_values

    # main loop: Prediction + Estimation
    for k in range(len(measurements)):
        # Step 1: Predict State and Error Covariance
        x_pred = A @ x
        P_pred = A @ P @ A.T + Q

        # Step 2: Compute the Kalman Gain
        K = P_pred @ H.T @ np.linalg.inv(H @ P_pred @ H.T + R)

        # Step 3: Compute the Estimate
        x = x_pred + K @ (measurements[k] - H @ x_pred)

        # Step 4: Compute the Error Covariance
        P = P_pred - K @ H @ P_pred

        # add current estimate to array for tracking
        x_estimates.append(x.copy())
        K_values.append(K.copy())
        P_values.append(P.copy())
    
    return {
            "final_estimate": x, 
            "estimates_over_time": x_estimates, 
            "kalman_gains": K_values, 
            "error_covariances": P_values
        }